In [83]:
# !pip install gensim

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdfplumber
from pdf2docx import Converter
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import docx2txt
from docx import Document
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import os
import glob
import win32com.client
import PyPDF2
#import camelot
import fitz
import sys
from pdfrw import PdfReader
import operator
import wx
from fuzzywuzzy import process
from nltk import ngrams
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.parsing.preprocessing import remove_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VEERANI2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\VEERANI2\.conda\envs\ana_envi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)



<h2> using pdfplumber </h2>

In [2]:
# with pdfplumber.open(r'noccodes.pdf') as pdf:
#     first_page = pdf.pages[487]
#     print(first_page.extract_text())

<h2> using pdfminer </h2>

In [3]:
noc_codes=pd.read_csv("noc_names.csv", encoding= 'unicode_escape',dtype = {'NOC Code': str,'Code Division': str,
                                                      'Code Name': str})

In [4]:
noc_codes.tail()

,NOC Code,Code Division,Code Name
496,9616,D,Labourers in textile processing
497,9617,D,Labourers in food and beverage processing
498,9618,D,Labourers in fish and seafood processing
499,9619,D,"Other labourers in processing, manufacturing a..."
500,NaN,NaN,NaN


In [5]:
#Extracting headings and paragraphs
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=operator.itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

def headings(doc):
    font_counts_, styles_=fonts(doc)
    size_tag_=font_tags(font_counts_, styles_)
    header_para_=headers_para(doc,size_tag_)
    headings=list()
    for val in header_para_:
        if "<h" in val:
            headings.append(val)
    val1=[sub.split("|") for sub in headings]
    flat_list = [item for sublist in val1 for item in sublist]
    df = pd.DataFrame(flat_list)
    return df

def sent_tags(doc):
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
    #                 block_string = list(map(lambda st: str.replace(st, ",", " and "), block_string)) 
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [6]:
doc = fitz.open('noccodes_edited.pdf')

In [7]:
# doc

In [8]:
font_counts_, styles_=fonts(doc)
size_tag_=font_tags(font_counts_, styles_)
header_para_=headers_para(doc,size_tag_)

In [9]:
# header_para_

In [10]:
md = [idx for idx, s in enumerate(header_para_) if "<h3>" in s]

In [11]:
header_para_[md[1]]

'<h3>00\xa0 Senior management occupations|'

In [12]:
len(header_para_) - 1

10274

In [13]:
md.append(len(header_para_) - 1)

In [14]:
md

[8,
 12,
 16,
 20,
 30,
 31,
 41,
 60,
 78,
 96,
 115,
 134,
 138,
 142,
 162,
 181,
 199,
 217,
 221,
 241,
 260,
 279,
 296,
 309,
 318,
 335,
 343,
 361,
 380,
 398,
 402,
 420,
 424,
 443,
 471,
 480,
 498,
 502,
 530,
 548,
 567,
 571,
 589,
 597,
 616,
 620,
 638,
 656,
 674,
 678,
 682,
 701,
 705,
 724,
 728,
 747,
 765,
 769,
 786,
 790,
 794,
 813,
 822,
 849,
 853,
 873,
 877,
 895,
 899,
 917,
 935,
 953,
 957,
 977,
 995,
 999,
 1003,
 1007,
 1016,
 1027,
 1045,
 1072,
 1091,
 1095,
 1114,
 1132,
 1160,
 1164,
 1168,
 1186,
 1203,
 1211,
 1229,
 1248,
 1252,
 1272,
 1290,
 1310,
 1329,
 1347,
 1364,
 1383,
 1403,
 1407,
 1426,
 1444,
 1452,
 1466,
 1484,
 1493,
 1511,
 1530,
 1534,
 1538,
 1557,
 1577,
 1596,
 1615,
 1635,
 1639,
 1643,
 1663,
 1682,
 1700,
 1718,
 1722,
 1741,
 1760,
 1764,
 1783,
 1802,
 1821,
 1839,
 1843,
 1852,
 1881,
 1890,
 1903,
 1907,
 1926,
 1935,
 1954,
 1958,
 1977,
 1997,
 2017,
 2036,
 2053,
 2071,
 2076,
 2080,
 2084,
 2093,
 2105,
 2133,
 2

In [15]:
d={}
for i in range(0,len(md)):
#     print(i)
    d["string{0}".format(i)] = ''.join(header_para_[md[i-1]:md[i]])

In [16]:
keys_=d.keys()
# keys_

In [17]:
d1={}
for val in keys_:
#     print(val)
    d1[d[val].split('|', 1)[0]]=d[val]
#     del d[val] 

In [18]:
d1.keys()

dict_keys(['', '<h3>0\xa0 Management occupations', '<h3>00\xa0 Senior management occupations', '<h3>001\xa0 Legislators and senior management', '<h3>0011\xa0 Legislators', '<h3>0 Management occupations', '<h3>Classification Description', '<h3>0012\xa0 Senior government managers and officials', '<h3>0013\xa0 Senior managers - financial, communications and other business services', '<h3>0014\xa0 Senior managers - health, education, social and community services and membership', '<h3>0015\xa0 Senior managers - trade, broadcasting and other services, n.e.c.', '<h3>0016\xa0 Senior managers - construction, transportation, production and utilities', '<h3>01-05\xa0 Specialized middle management occupations', '<h3>011\xa0 Administrative services managers', '<h3>0111\xa0 Financial managers', '<h3>0112\xa0 Human resources managers', '<h3>0113\xa0 Purchasing managers', '<h3>0114\xa0 Other administrative services managers', '<h3>012\xa0 Managers in financial and business services', '<h3>0121\xa0 In

In [19]:
d1 = {x.replace('<h3>', ''): v 
     for x, v in d1.items()}
d1 = {x.replace('\xa0', ''): v 
     for x, v in d1.items()}

In [20]:
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return " "
#         return "The strings are {} edits away".format(distance[row][col])

In [85]:
with open('resume_text.txt') as f:
    contents = f.read()
    
def removearticlesandstopwords(text):

    re.sub('\s+(a|an|and|the)(\s+)', '\2', text)
    text = remove_stopwords(text)
        
    return text
    
contents=removearticlesandstopwords(contents)


In [86]:
contents

'+91 88974 64355 +91 88974 64355 nithya.v@outlook.com nithya.v@outlook.com Key Skills Product Ownership Stakeholder Management Data Analytics Natural Language Processing Artificial Intelligence Forecasting Techniques Key Skills Product Ownership Stakeholder Management Data Analytics Natural Language Processing Artificial Intelligence Forecasting Techniques Certifications Six Sigma Yellow Belt Certification field Lean Six Sigma Quality Assurance Control, 2017. BPE Certified Certifications Six Sigma Yellow Belt Certification field Lean Six Sigma Quality Assurance Control, 2017. BPE Certified Novartis (2018 - Current) Solution Design Expert – ICS Owned deployed data science products machine learning models Enterprise Content Management space Novartis Was Product Owner multiple products including PrivacyAI, in-house document redaction anonymization tool built basis self-learning deep learning models creating application resulted cost benefits $6.4M period 5 years, 76% lower costs current s

In [22]:
# for val in d1.keys():
#     Distance = levenshtein_ratio_and_distance(contents,d1[val])
# #     print(Distance)
#     Ratio = levenshtein_ratio_and_distance(contents,d1[val],ratio_calc = True)
#     print("% match between resume and "+val+": "+ Ratio)

In [23]:
# list_=[]
# for val in d1.keys():
#     list_.append(val+" "+d1[val])

In [24]:
# list1=list_[10:20]

In [25]:
# Ratios = process.extract(contents,list1)
# print(Ratios)

In [26]:
# highest = process.extractOne(contents,list_)
# print(highest)

In [63]:
def cos_dist(text1, text2, q=3, pad=True):
    from scipy import spatial
    text1, text2 = create_ngram(text1, text2, n=q, pad=pad)
    
    full_text = list(text1.union(text2))
    v1 = [(lambda x: 1 if x in text1 else 0)(x) for x in full_text]
    v2 = [(lambda x: 1 if x in text2 else 0)(x) for x in full_text]    
    return spatial.distance.cosine(v1, v2)

In [64]:
def ngram(text, n=3, pad=True):
    text = text.strip()
    if pad:
        text = " %s " % text
    return set([text[i:i+n] for i in range(len(text)-n+1)])

def create_ngram(text1, text2, n=3):
    return list(ngrams(text1.split(), n=n)), list(ngrams(text2.split(), n=n))

In [87]:
contents

'+91 88974 64355 +91 88974 64355 nithya.v@outlook.com nithya.v@outlook.com Key Skills Product Ownership Stakeholder Management Data Analytics Natural Language Processing Artificial Intelligence Forecasting Techniques Key Skills Product Ownership Stakeholder Management Data Analytics Natural Language Processing Artificial Intelligence Forecasting Techniques Certifications Six Sigma Yellow Belt Certification field Lean Six Sigma Quality Assurance Control, 2017. BPE Certified Certifications Six Sigma Yellow Belt Certification field Lean Six Sigma Quality Assurance Control, 2017. BPE Certified Novartis (2018 - Current) Solution Design Expert – ICS Owned deployed data science products machine learning models Enterprise Content Management space Novartis Was Product Owner multiple products including PrivacyAI, in-house document redaction anonymization tool built basis self-learning deep learning models creating application resulted cost benefits $6.4M period 5 years, 76% lower costs current s

In [88]:
create_ngram(contents,removearticlesandstopwords(d1['2173 Software engineers and designers']))

([('+91', '88974', '64355'),
  ('88974', '64355', '+91'),
  ('64355', '+91', '88974'),
  ('+91', '88974', '64355'),
  ('88974', '64355', 'nithya.v@outlook.com'),
  ('64355', 'nithya.v@outlook.com', 'nithya.v@outlook.com'),
  ('nithya.v@outlook.com', 'nithya.v@outlook.com', 'Key'),
  ('nithya.v@outlook.com', 'Key', 'Skills'),
  ('Key', 'Skills', 'Product'),
  ('Skills', 'Product', 'Ownership'),
  ('Product', 'Ownership', 'Stakeholder'),
  ('Ownership', 'Stakeholder', 'Management'),
  ('Stakeholder', 'Management', 'Data'),
  ('Management', 'Data', 'Analytics'),
  ('Data', 'Analytics', 'Natural'),
  ('Analytics', 'Natural', 'Language'),
  ('Natural', 'Language', 'Processing'),
  ('Language', 'Processing', 'Artificial'),
  ('Processing', 'Artificial', 'Intelligence'),
  ('Artificial', 'Intelligence', 'Forecasting'),
  ('Intelligence', 'Forecasting', 'Techniques'),
  ('Forecasting', 'Techniques', 'Key'),
  ('Techniques', 'Key', 'Skills'),
  ('Key', 'Skills', 'Product'),
  ('Skills', 'Produc

In [89]:
def qgrams(text1, text2, q=3):
    text11, text22 = create_ngram(contents,removearticlesandstopwords(d1['2173 Software engineers and designers']), n=q)
    return len(set(text11).intersection(text22)),set(text11).intersection(text22)

In [92]:
len_,vals=qgrams(contents,d1['2173 Software engineers and designers'], q=2) # 3
len_

0

In [91]:
vals

{('-',),
 ('application',),
 ('applications',),
 ('data',),
 ('document',),
 ('including',),
 ('interactive',),
 ('lead',),
 ('models',),
 ('process',),
 ('program',),
 ('science',),
 ('software',),
 ('teams',),
 ('|',),
 ('–',)}

In [93]:
def jaccard_dist(text1, text2, q=3, pad=True):
    text11, text22 = create_ngram(contents,removearticlesandstopwords(d1['2173 Software engineers and designers']), n=q)
    
    full_text = list(text11.union(text22))
    agree_tot = len(text11.intersection(text22))
    return 1 - agree_tot/float(len(full_text)), text11.intersection(text22)


In [94]:
a,b=jaccard_dist(contents,d1['2173 Software engineers and designers'], q=15, pad=False) #0.5

AttributeError: 'list' object has no attribute 'union'

In [ ]:
b

In [ ]:
text11.intersection(text22)

In [ ]:
def tversky_index(text1, text2, a=None, b=None, q=3, pad=True):
    text11, text22 = create_ngram(text1, text2, n=q, pad=pad)
    agree_tot = len(text11.intersection(text22))
    v1 = len(text11) - agree_tot
    v2 = len(text22) - agree_tot
    
    if a != None and b != None:
        a = a/float(a+b)
        b = b/float(a+b)
    elif a <= 1.0 and a >= 0.0:
        b = 1-a
    elif b <= 1.0 and b >= 0.0:
        a = 1-b
    else:
        a = 0.5
        b = 0.5        
    return float(agree_tot)/(agree_tot+a*v1+b*v2)
    


In [ ]:
tversky_index(contents,d1['2173 Software engineers and designers'], a=0.5, q=15, pad=False) # 0.6666666

In [ ]:
text

In [ ]:
cos_dist(contents,d1['2173 Software engineers and designers'] , q=2, pad=False)

In [ ]:
d1['2173 Software engineers and designers']